In [101]:
import pandas as pd
import numpy as np


----
Read in collected data


In [102]:
mdf = pd.read_csv("../../data/raw/meteo_weather.csv")
fdf = pd.read_csv("../../data/processed/wildfire.csv")
ndf = pd.read_csv("../../data/raw/weather.csv")


Drop unnamed column


In [103]:
mdf.drop(columns="Unnamed: 0", inplace=True)
ndf.drop(columns="Unnamed: 0", inplace=True)


In [104]:
mdf.head()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,lat,long,id,index,station
0,2020-07-22,35.4,30.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.195810,-111.348700,0,NaN,NaN
1,2020-07-23,30.6,27.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.195810,-111.348700,0,NaN,NaN
2,2020-07-19,25.9,14.4,35.6,0.0,NaN,NaN,8.0,NaN,1013.1,NaN,40.602563,-115.719777,1,NaN,NaN
3,2020-07-20,26.3,13.3,35.6,0.0,NaN,NaN,5.7,NaN,1012.4,NaN,40.602563,-115.719777,1,NaN,NaN
4,2020-07-21,25.3,15.0,33.9,0.0,NaN,359.0,9.0,NaN,1010.6,NaN,40.602563,-115.719777,1,NaN,NaN


> Meteo is missing too many values and data id missing 15k from fire data and can not be used

Drop dups from NASA dataset and keep fire start date row

Generate new columns
- Precep sum of all days
- Wind mean
- Humidity mean
- temp average

In [105]:
ndf.rename(columns={'ID': 'pid'}, inplace=True)

In [106]:
ndf_mean = ndf.groupby("pid").mean()[
    ["WS2M", "WS2M_MAX", "WS10M", "WS10M_MAX", "QV2M", "T2M"]
]

ndf_mean.rename(
    columns={
        "WS2M": "wind_speed_2m_mean",
        "WS2M_MAX": "wind_speed_2m_max_mean",
        "WS10M": "wind_speed_10m_mean",
        "WS10M_MAX": "wind_speed_10m_max_mean",
        "QV2M": "humidity_mean",
        "T2M": "temp_2m_mean",
    },
    inplace=True,
)

ndf_sum = ndf.groupby("pid").sum()[["PRECTOTCORR"]]
ndf_sum.rename(
    columns={
        "PRECTOTCORR": "rain_sum",
    },
    inplace=True,
)

print(ndf_sum.head(1))
ndf_mean.head(1)


     rain_sum
pid          
0         2.6


,wind_speed_2m_mean,wind_speed_2m_max_mean,wind_speed_10m_mean,wind_speed_10m_max_mean,humidity_mean,temp_2m_mean
pid,,,,,,
0,1.41,3.09,1.895,4.18,13.21,29.9


In [107]:
first_date_fire_weather = ndf.drop_duplicates(subset="pid", keep="first")
first_date_fire_weather.drop(columns=["YEAR", "MO", "DY", "LAT", "LONG"], inplace=True)
first_date_fire_weather.set_index("pid", inplace=True)

first_date_fire_weather.rename(
    columns={
        "PRECTOTCORR": "rain",
        "WS2M": "wind_speed_2m",
        "WS2M_MAX": "wind_speed_2m_max",
        "WS10M": "wind_speed_10m",
        "WS10M_MAX": "wind_speed_10m_max",
        "QV2M": "humidity",
        "T2M": "temp_2m",
        "T2M_MAX": "temp_2m_max",
        "GWETTOP": "surface_soil_wetness_5cm_below",
        "GWETPROF": "surface_soil_wetness_to_bedrock",
    },
    inplace=True,
)

first_date_fire_weather.head()


C:\Users\btoli\AppData\Local\Temp\ipykernel_23028\980918030.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_date_fire_weather.drop(columns=["YEAR", "MO", "DY", "LAT", "LONG"], inplace=True)
C:\Users\btoli\AppData\Local\Temp\ipykernel_23028\980918030.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_date_fire_weather.rename(


,temp_2m,temp_2m_max,humidity,rain,wind_speed_2m,wind_speed_2m_max,wind_speed_10m,wind_speed_10m_max,surface_soil_wetness_5cm_below,surface_soil_wetness_to_bedrock
pid,,,,,,,,,,
0,32.63,38.06,12.02,0.16,1.26,2.77,1.71,3.84,0.12,0.38
1,24.80,33.19,5.92,0.00,1.76,3.42,2.61,4.52,0.31,0.37
2,18.91,26.46,7.93,3.44,2.50,3.65,4.16,5.82,0.26,0.35
3,5.48,16.30,2.56,0.00,1.25,2.98,1.88,4.02,0.17,0.34
4,31.50,39.08,8.91,0.26,2.30,4.45,3.26,5.90,0.15,0.36


Merge mean and original weather data

In [108]:
first_date_fire_weather = first_date_fire_weather.merge(
    ndf_mean, how="inner", left_index=True, right_index=True
)
first_date_fire_weather = first_date_fire_weather.merge(
    ndf_sum, how="inner", left_index=True, right_index=True
)


In [109]:
first_date_fire_weather.columns

Index(['temp_2m', 'temp_2m_max', 'humidity', 'rain', 'wind_speed_2m',
       'wind_speed_2m_max', 'wind_speed_10m', 'wind_speed_10m_max',
       'surface_soil_wetness_5cm_below', 'surface_soil_wetness_to_bedrock',
       'wind_speed_2m_mean', 'wind_speed_2m_max_mean', 'wind_speed_10m_mean',
       'wind_speed_10m_max_mean', 'humidity_mean', 'temp_2m_mean', 'rain_sum'],
      dtype='object')

Merge fire and with NASA weather data

In [110]:
fdf.set_index('id', inplace=True)

In [111]:
merged_df = fdf.merge(
    first_date_fire_weather, how="left", left_index=True, right_index=True
)


In [112]:
merged_df.isnull().sum()

X                                  0
Y                                  0
ContainmentDateTime                0
ControlDateTime                    0
DailyAcres                         0
DiscoveryAcres                     0
FireCause                          0
FireDiscoveryDateTime              0
IncidentTypeCategory               0
IncidentTypeKind                   0
InitialLatitude                    0
InitialLongitude                   0
IrwinID                            0
LocalIncidentIdentifier            0
POOCounty                          0
POODispatchCenterID                0
POOFips                            0
POOState                           0
UniqueFireIdentifier               0
temp_2m                            4
temp_2m_max                        4
humidity                           4
rain                               4
wind_speed_2m                      4
wind_speed_2m_max                  4
wind_speed_10m                     4
wind_speed_10m_max                 4
s

drop missing 4 rows

In [113]:
merged_df.dropna(inplace=True)

Lowercase col names

In [114]:
merged_df.columns = merged_df.columns.str.lower()

In [115]:
merged_df.head(1)

,x,y,containmentdatetime,controldatetime,dailyacres,discoveryacres,firecause,firediscoverydatetime,incidenttypecategory,incidenttypekind,...,wind_speed_10m_max,surface_soil_wetness_5cm_below,surface_soil_wetness_to_bedrock,wind_speed_2m_mean,wind_speed_2m_max_mean,wind_speed_10m_mean,wind_speed_10m_max_mean,humidity_mean,temp_2m_mean,rain_sum
id,,,,,,,,,,,,,,,,,,,,,
0,-111.348611,33.195755,2020-07-23 05:29:59+00:00,2020-07-23 05:29:59+00:00,8.0,2.5,Human,2020-07-22 21:51:00+00:00,WF,FI,...,3.84,0.12,0.38,1.41,3.09,1.895,4.18,13.21,29.9,2.6


In [116]:
merged_df.reset_index().to_csv("../../data/processed/wildfire_weather.csv", index=False)


----
Add extra dataset

In [117]:
extra_weather = pd.read_csv('../../data/raw/nasa_weather_extra.csv')

extra_weather.head(1)

,YEAR,MO,DY,T2MDEW,T2MWET,RH2M,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_PAR_TOT,ALLSKY_SFC_UV_INDEX,LAT,LONG,ID
0,2020,7,19,3.02,13.91,27.31,155.74,135.5,2.55,40.602563,-115.719777,1


In [118]:
extra_weather.drop(columns=['YEAR', 'MO', 'DY', 'LAT', 'LONG'], inplace=True)

In [119]:
extra_weather.drop_duplicates(keep='first', inplace=True)

In [120]:
extra_weather.set_index('ID', inplace=True)

In [121]:
# extra_weather.rename(columns={
#   'T2MDEW': 'dew_frost_point_2m',
#   'T2MWET': 'wet_bulb_temp_2m',
#   'RH2M': 'relative_humidity',
#   'CLRSKY_SFC_PAR_TOT': 'clear_sky_photosynthetically_active_radiation',
#   'ALLSKY_SFC_PAR_TOT': 'all_sky_photosynthetically_active_radiation',
#   'ALLSKY_SFC_UV_INDEX': 'all_sky_surface_uv_index'
# }, inplace=True)

extra_weather.head(1)

,dew_frost_point_2m,wet_bulb_temp_2m,relative_humidity,clear_sky_photosynthetically_active_radiation,all_sky_photosynthetically_active_radiation,all_sky_surface_uv_index
ID,,,,,,
1,3.02,13.91,27.31,155.74,135.5,2.55


In [122]:
merged_df = merged_df.merge(extra_weather, how='left', left_index=True, right_index=True)

In [123]:
merged_df.dropna(inplace=True)

In [124]:
merged_df.shape

(39219, 42)

In [125]:
merged_df.columns

Index(['x', 'y', 'containmentdatetime', 'controldatetime', 'dailyacres',
       'discoveryacres', 'firecause', 'firediscoverydatetime',
       'incidenttypecategory', 'incidenttypekind', 'initiallatitude',
       'initiallongitude', 'irwinid', 'localincidentidentifier', 'poocounty',
       'poodispatchcenterid', 'poofips', 'poostate', 'uniquefireidentifier',
       'temp_2m', 'temp_2m_max', 'humidity', 'rain', 'wind_speed_2m',
       'wind_speed_2m_max', 'wind_speed_10m', 'wind_speed_10m_max',
       'surface_soil_wetness_5cm_below', 'surface_soil_wetness_to_bedrock',
       'wind_speed_2m_mean', 'wind_speed_2m_max_mean', 'wind_speed_10m_mean',
       'wind_speed_10m_max_mean', 'humidity_mean', 'temp_2m_mean', 'rain_sum',
       'dew_frost_point_2m', 'wet_bulb_temp_2m', 'relative_humidity',
       'clear_sky_photosynthetically_active_radiation',
       'all_sky_photosynthetically_active_radiation',
       'all_sky_surface_uv_index'],
      dtype='object')

In [126]:
merged_df[merged_df['dailyacres'] > 1].shape

(39219, 42)

Drop -999.00

In [129]:
merged_df = merged_df[(merged_df != -999.00).all(axis=1)]

In [132]:
merged_df.to_csv('../../data/cleaned/bigger_wildfire_weather_extra.csv', index=False)